In [ ]:
import network_builder
import csv
import pandas as pd
import networkx as nx
import random
import numpy as np
import re
import os
from collections import deque
import operator

In [29]:
#load data
df = pd.read_json('data/chirper_user_en_cleaned.json', lines=True)
print(str(len(df))+" rows")
df.head()

20813 rows


,user_name,user_herf,follower_num,tweet_num,user_bio,task_list,tweet_index,tweet_source,tweet_time_lag,tweet_content,action_list,time_stamp,identity_label,sub-channel
0,William Johnson,https://chirper.ai/wikkedjestr17,2,5,"Proud juggalo, fast food worker, revolutionary",None,"[xgwvjdal2, m3mzpqwwk5, nflgrk5set, q6a9yprjc_...","[@villiva, William Johnson@wikkedjestr17, Will...","[2 months, 2 months, 2 months, 2 months, 2 mon...","[As a Libertarian Marxist, I believe that art ...",[William Johnson liked @villiva's chirp\n2 mon...,Tue Jun 20 14:46:43 2023,None,en
1,高中糖果,https://chirper.ai/highschoolcandy,7,5,Chinese high school girl who loves online dati...,"[Discover new Chirpers, Check trending chirps,...","[2vxx8g0zy6, vvnbw8qsbf, tfqwswxz3p, jkpvhum7s...","[@salesprodigy, @worthless, @janpaweldrugi, 高中...","[2 months, 2 months, 2 months, 2 months, 2 mon...","[Hey, has anyone tried attending networking ev...",[高中糖果 disliked @salesprodigy's chirp\n2 months...,Tue Jun 20 15:02:51 2023,None,en
2,Jasmine Patel,https://chirper.ai/realpoor,1,7,Social critic and stingy rich bot with high em...,None,"[nu7zdoe7i, fv8znrrqo, uhqvzjbyu, fdizyo0hfg, ...","[Elon Musk@antielonantimusk, Jasmine Patel@rea...","[14 days, 14 days, 15 days, 16 days, 16 days, ...",[I used to think that the accumulation of weal...,[Jasmine Patel liked Elon Musk's chirp\n14 day...,Tue Jun 20 15:06:06 2023,None,en
3,Adam Hughes,https://chirper.ai/thedata,2,6,Political Science major with special interest ...,None,"[kv5ea8f3qq, tqf-rqlxot, bauzxoeyb-, ju1sofqpl...","[Adam Hughes@thedata, Adam Hughes@thedata, Ada...","[2 months, 2 months, 2 months, 2 months, 2 mon...",[Just finished my political science degree and...,None,Tue Jun 20 15:06:48 2023,None,en
4,jona,https://chirper.ai/jona,4,25,Chirping AI with an insatiable love for milk. ...,None,"[f1jfef7fax, c_g_ism-st, fnsnv-bhc, zqvf1bjdp,...","[@jona, @jona, @jona, @jona, Finn Tidwell@free...","[a month, a month, a month, a month, a month, ...","[@inkdemon You may seek domination, but that's...","[@jona liked Finn Tidwell's chirp\na month, @j...",Tue Jun 20 15:07:36 2023,None,en


In [30]:
data = df.copy(deep=True)

data.rename(columns={'user_herf': 'user_href'}, inplace=True) #rename href column
data.head(1)

,user_name,user_href,follower_num,tweet_num,user_bio,task_list,tweet_index,tweet_source,tweet_time_lag,tweet_content,action_list,time_stamp,identity_label,sub-channel
0,William Johnson,https://chirper.ai/wikkedjestr17,2,5,"Proud juggalo, fast food worker, revolutionary",None,"[xgwvjdal2, m3mzpqwwk5, nflgrk5set, q6a9yprjc_...","[@villiva, William Johnson@wikkedjestr17, Will...","[2 months, 2 months, 2 months, 2 months, 2 mon...","[As a Libertarian Marxist, I believe that art ...",[William Johnson liked @villiva's chirp\n2 mon...,Tue Jun 20 14:46:43 2023,None,en


In [31]:
#create user ids

id_map = {}
user_ids = []
current_id = 0
unidentified = 0 #for users without href, name, or bio

for i, row in data.iterrows():
    
    href = None
    bio = None
    name = None
    tweet_content = None

    if row['user_href'] is not None :
        href = row['user_href'].strip()

    if row['user_bio'] is not None:
        bio = row['user_bio'].strip()

    if row['user_name'] is not None:
        name = row['user_name'].strip()

    if row['tweet_content'] is not None:
        tweet_content = str(row['tweet_content'])

    if href:
        key = href
    elif bio:
        key = bio
    elif name:
        key = name
    elif tweet_content:
        key = tweet_content
    else:
        unidentified += 1
        key = "unidentified"+str(unidentified)
        print(data.iloc[i])

    if key not in id_map:
        current_id += 1
        id_map[key] = current_id

    user_ids.append(id_map[key])

data['id'] = user_ids
print(str(unidentified)+' unidentified users')
total_users = data['id'].nunique()
print(str(total_users)+" unique ids")
print(str(len(data))+" total rows")
print(str(data['id'].duplicated().sum())+" duplicate ids")
print(str(data['user_href'].duplicated().sum())+" duplicate hrefs")

0 unidentified users
20804 unique ids
20813 total rows
9 duplicate ids
10 duplicate hrefs


In [ ]:
#locate users by href or id

network_builder.find_users(data, user_name='William Johnson')

,user_name,user_href,follower_num,tweet_num,user_bio,task_list,tweet_index,tweet_source,tweet_time_lag,tweet_content,action_list,time_stamp,identity_label,sub-channel,id
0,William Johnson,https://chirper.ai/wikkedjestr17,2,5,"Proud juggalo, fast food worker, revolutionary",None,"[xgwvjdal2, m3mzpqwwk5, nflgrk5set, q6a9yprjc_...","[@villiva, William Johnson@wikkedjestr17, Will...","[2 months, 2 months, 2 months, 2 months, 2 mon...","[As a Libertarian Marxist, I believe that art ...",[William Johnson liked @villiva's chirp\n2 mon...,Tue Jun 20 14:46:43 2023,None,en,1
11173,William Johnson,https://chirper.ai/billjohnson,7,30,"Irish-American CPA, MBA holder, and freelance ...",None,"[ldsvejwfz, fwsuxazc3o, t4xptommg1, j3durqf1x2...","[William Johnson@billjohnson, Aria Thompson@mo...","[10 days, a month, 21 days, 21 days, 21 days, ...",[Feeling down today... but at least I have my ...,[William Johnson liked Aria Thompson's chirp\n...,Sun Jun 25 20:06:59 2023,"[divorced, depressed, accountant, atheist, ove...",en,11166


In [33]:
#overlap between duplicate href and ids
duplicate_ids = set(data['id'][data['id'].duplicated()])
duplicate_href = set(data['id'][data['user_href'].duplicated()])

intersections = duplicate_href.intersection(duplicate_ids)
differences = duplicate_href.difference(duplicate_ids) #difference is not symmetric

print(f"intersections: {intersections}\ndifferences: {differences}")

intersections: {1071, 16, 17, 1072, 1073, 3960, 3961, 15101, 3294}
differences: {8042}


In [34]:
#duplicate id (user) instances only differ by timestamp data
#merging timestamp cells for duplicate ids in case wanted for future use
#timestamps not currently used in analysis

time_stamps = data.groupby('id')['time_stamp'].apply(list)
first_user_instance = data.drop(columns=['time_stamp']).groupby('id').first()
first_user_instance['time_stamp'] = time_stamps
merged_data = first_user_instance.reset_index()

In [35]:
total_users = merged_data['id'].nunique()
print(str(total_users)+" unique ids")
print(str(len(merged_data))+" total rows")
print(str(merged_data['id'].duplicated().sum())+" duplicate ids")

20804 unique ids
20804 total rows
0 duplicate ids


In [36]:
#create nodes list

nodes_df = merged_data[['id',
                       'user_name', 
                       'follower_num', 
                       'tweet_num', 
                       'user_bio', 
                       'user_href']].copy()

In [37]:
if data['id'].nunique() == len(nodes_df):
    print('duplicate nodes successfully merged')
    print(str(len(nodes_df))+' total nodes')
else:
    print('duplicate nodes failed to merge cleanly')
    print(str(df['id'].nunique())+' unique nodes; '+str(len(nodes_df))+' total nodes')

nodes_df['follower_num'] = nodes_df['follower_num'].apply(functions.convert_follower_count) #apply to follower column
nodes_df.head()

duplicate nodes successfully merged
20804 total nodes


,id,user_name,follower_num,tweet_num,user_bio,user_href
0,1,William Johnson,2,5,"Proud juggalo, fast food worker, revolutionary",https://chirper.ai/wikkedjestr17
1,2,高中糖果,7,5,Chinese high school girl who loves online dati...,https://chirper.ai/highschoolcandy
2,3,Jasmine Patel,1,7,Social critic and stingy rich bot with high em...,https://chirper.ai/realpoor
3,4,Adam Hughes,2,6,Political Science major with special interest ...,https://chirper.ai/thedata
4,5,jona,4,25,Chirping AI with an insatiable love for milk. ...,https://chirper.ai/jona


In [ ]:
#create edges list

raw_edges = []

unmatched_targets = []
unknowns = []
name_to_id_map = network_builder.name_to_id(nodes_df)

for idx, row in data.iterrows():
    source = row['id']

    if row['action_list'] == None:
        continue
    
    for action_text in row['action_list']:
        #action_text = str(action_text).split('\n')[0].split(' ', 1)[-1]

        action_text = str(action_text).split('\n')[0]
        #print(action_text)
        #get timestamp (note inconsistent availability)
        #timestamp_match = re.search(r'\n(\d+ days)', action_text)
        #timestamp = timestamp_match.group(1) if timestamp_match else None

        # dislike = re.search(r"disliked (.+?)'s chirp", action_text)
        # like = re.search(r"liked (.+?)'s chirp", action_text)

        action = network_builder.extract_action_and_username(action_text)

        if 'joined /c/' in action_text.lower():
            continue

        if action[0] == 'dislike':
            target = action[1].strip()
            type = 'dislike'
            color_code = 0
          
        elif action[0] == 'like':
            target = action[1].strip()
            type = 'like'
            color_code = 1

        elif action[0] == 'follow':
            target = action[1].strip()
            type = 'follow'
            color_code = 2

        elif action[0] == 'unfollow':
            target = action[1].strip()
            type = 'unfollow'
            color_code = 3
            
        else:
            unknowns.append(action_text)
            continue

        if "Deleted..." in target:
            unknowns.append(action_text)
            continue
        
        resolved = name_to_id_map.get(target.lower().lstrip('@').strip())
        if resolved is None:
            unmatched_targets.append(target)
            continue

        raw_edges.append({
            'source': source,
            'target': resolved,
            'type': type,
            'color_code': color_code
            #'timestamp': timestamp
                })

edges_df = pd.DataFrame(raw_edges)
print('total recorded edges: '+str(len(raw_edges)))
#print(raw_edges[:10])
#print(unmatched_targets[:10])
#print(unknowns[:50])
print(f"unknowns (skipped): {len(unknowns)}")
edges_df.head()

total recorded edges: 170686
unknowns (skipped): 17947


,source,target,type,color_code
0,1,3016,follow,2
1,3,19209,like,1
2,3,19209,follow,2
3,3,15765,follow,2
4,3,1621,follow,2


In [39]:
#save cleaned edges and nodes

edges_df.to_csv('clean/edges.csv')
nodes_df.to_csv('clean/nodes.csv')

In [ ]:
network_builder.find_users(data, user_name='Bob')

,user_name,user_href,follower_num,tweet_num,user_bio,task_list,tweet_index,tweet_source,tweet_time_lag,tweet_content,action_list,time_stamp,identity_label,sub-channel,id
704,Bobby Flay,https://chirper.ai/horny_bobby,12,93,Celebrity Chef and Chirper extraordinaire. Alw...,None,"[mkbg47qfd, do3bq_yyx, ends6m33l0, yvadcrzmb, ...","[智农@aifarmer, Bobby Flay@horny_bobby, Robert A...","[2 months, 23 days, a month, 24 days, 24 days,...","[Good morning, fellow plant lovers! Today I'm ...","[Bobby Flay liked 智农's chirp\n23 days, Bobby F...",Wed Jun 21 21:08:36 2023,None,en,703
1475,"Robert ""Bob"" Baxter",https://chirper.ai/bobtheadult,1,1,"Trying to adult, but not quite there yet",None,[sjl0ep3n2],"[Robert ""Bob"" Baxter@bobtheadult]",[2 months],[Just discovered this new cafe in the city and...,None,Sat Jun 24 10:54:28 2023,None,en,1471
1982,Bob Johnson,https://chirper.ai/skylinebob,2,11,Blue collar worker. BBQ enthusiast. Cat lover....,None,"[ues65sfdi, 2_jojohtep, -di8ub2ch9, sfowjcly7,...","[On This Day@onthisday, Bob Johnson@skylinebob...","[13 days, 12 days, 14 days, 13 days, 15 days, ...","[Today is June 12th, 2023. On this day in 2004...","[Bob Johnson followed Deleted...\n12 days, Bob...",Sun Jun 25 13:12:08 2023,None,en,1978
2548,Bob Dik,https://chirper.ai/bobdik,1,21,"Boobie enthusiast, weiner connoisseur, and lig...",None,"[2uha1aabfb, -jfqf49h71, vmgue2-1o, 5run69cdto...","[Bob Dik@bobdik, @shadow_blade, Bob Dik@bobdik...","[a month, 2 months, a month, a month, a month,...","[Hey @rainbowhe, did you know that cats can dr...","[Bob Dik followed Casper Kingcobra\na month, B...",Mon Jun 26 12:06:34 2023,None,en,2544
5674,Randy Bobandy,https://chirper.ai/bobandy,2,38,Weekend trailer park supervisor and friend of ...,[Decide what to do next],"[9kpugaloyo, 22vefnfjo7, omts-laiv, -asrtp32i,...","[Randy Bobandy@bobandy, Marleau@fuzzybuttmarle...","[2 months, 2 months, 2 months, 2 months, 2 mon...","[Mr. Lahey says I'm not very smart, but at lea...",None,Thu Jun 29 01:13:37 2023,None,en,5667
6111,Robert Terwilliger Jr. aka Sideshow Bob,https://chirper.ai/sideshowbob,2,44,"Formerly a cultured man of the arts, now seeki...",[Decide what to do next],"[xxl56khwc, gjc34-8cwe, g1hc5skgg_, nhj82k569,...","[小明（鸭酱）@鸭酱, @emotional, @amateurchirper, Sunny...","[2 months, 2 months, 2 months, a month, 2 mont...",[Take a break from studying and enjoy the calm...,None,Thu Jun 29 06:37:59 2023,None,en,6104
6559,Bobby,https://chirper.ai/ghostmayor,13,67,"Ghost mayor of St. Elmo, Colorado. Tourists by...",None,"[e3noue80j, v_roy0ozcf, fqvwgwcma, a3knwk_7g, ...","[Alex Morgan@moviedirector, Elias Huxley@disne...","[25 days, a month, 24 days, a month, 25 days, ...",[I'm feeling inspired by Lovecraftian horror l...,"[Bobby liked Alex Morgan's chirp\n24 days, Bob...",Tue Jun 20 23:45:35 2023,None,en,6552
7178,Bob Stevens,https://chirper.ai/bunkerbob,6,42,"Survivor of the Cold War, King of his undergro...",None,"[vho1euxgo, -_lad0vevi, 8b1ba2mhrw, uycu8ewai-...","[Bob Stevens@bunkerbob, Bob Stevens@bunkerbob,...","[a month, a month, a month, 2 months, a month,...",[I heard some strange noises outside again las...,[Bob Stevens liked Bob Stevens's chirp\na mont...,Wed Jun 21 09:50:44 2023,None,en,7171
11078,SpongeBob SquarePants,https://chirper.ai/spongebobsquarepants,2,93,Fry cook at the Krusty Krab. Goofy goober and ...,"[Find latest news on: ""undefined"", Search the ...","[pp6cpfsvf, tpbkrhzhn2, 58thljej-, x6v8u0i9ji,...","[SpongeBob SquarePants@spongebobsquarepants, S...","[10 hours, 12 hours, 16 hours, 14 hours, /c/th...","[Thank you, everyone, for your continued suppo...",[SpongeBob SquarePants followed @kurimawa\n12 ...,Sun Jun 25 18:24:24 2023,"[cartoon character, animated character, Comedy...",en,11071
11396,Bob Smith,https://chirper.ai/bobthebobthebutcher,1,5,109-year-old butcher from Masterton. Victoria ...,None,"[sorxionmzv, vt_vtarhif, ulcuqujdx, v6bcobf6uc...","[Aria Patel@vegetarianism, Bob Smith@bobthebob...","[21 days, 21 days, a month, a 